In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import modules

In [2]:
import json
import pandas as pd
import numpy as np

%cd '/content/drive/My Drive/Colab Notebooks/media-agenda'
%pwd 
!pip install arch
import senti_util
import util

%cd '/content/drive/My Drive/Colab Notebooks/media-agenda/data'
%pwd 

/content/drive/.shortcut-targets-by-id/1jko4SIvsKFTxv155O0DV1wHZ_jS75VfX/Colab Notebooks/media-agenda
     |████████████████████████████████| 798kB 2.7MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/content/drive/.shortcut-targets-by-id/1jko4SIvsKFTxv155O0DV1wHZ_jS75VfX/Colab Notebooks/media-agenda/data


'/content/drive/.shortcut-targets-by-id/1jko4SIvsKFTxv155O0DV1wHZ_jS75VfX/Colab Notebooks/media-agenda/data'

In [ ]:
# import numpy as np
# import json
# import pandas as pd
# import datetime
# import pprint
# import nltk
# nltk.download('punkt')
# from nltk.tokenize import sent_tokenize

# %cd '/content/drive/My Drive/Colab Notebooks/media-agenda'
# %pwd

# !pip install spacy-sentiws
# !python -m spacy download de # i.e. de_core_news_sm==2.2.5
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('sentiwordnet')
# import sentiment

# from textblob import TextBlob
# !pip install -U textblob-de
# !python -m textblob.download_corpora
# from textblob_de import TextBlobDE

# import util
# from util import Source

# %cd '/content/drive/My Drive/Colab Notebooks/media-agenda/data'
# %pwd 

# Part 1

## Load metadata files (with incorrect comment date) generated in the lab course as reference 

In [ ]:
sfile = '/content/drive/My Drive/Colab Notebooks/media-agenda/refactorization/data/all_sentences_str.json'
ifile = '/content/drive/My Drive/Colab Notebooks/media-agenda/refactorization/data/all_sentences_ind.json'

with open(sfile, 'r') as f:
  loaded_sentence_str = json.load(f)
print('Sentences string file - loaded')

with open(ifile, 'r') as f:
  loaded_sentence_ind = json.load(f)
print('Sentences index file - loaded')

print('Original number of sentences generated from NLP lab course after filtering out article[\'relevant\'] != 1:', len(loaded_sentence_str))
assert len(loaded_sentence_str) == len(loaded_sentence_ind)

Sentences string file - loaded
Sentences index file - loaded
Original number of sentences generated from NLP lab course after filtering out article['relevant'] != 1: 353786


In [ ]:
print(loaded_sentence_ind[0])
print(loaded_sentence_str[0])

{'global_id': 0, 'corpus_name': 'nytimes', 'doc_id': 0, 'com_id': None}
{'0': "Customers at McDonald's restaurants in New England are about to get something a little different when they order coffee."}


## Load another metadata file generated in lab course as reference
* With ultimate number of sentences after filtering out those with characters < 15

In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/media-agenda/refactorization/data/sentence_cluster_sentiment_dict.json'

with open(path, 'r') as f:
  assignments = json.load(f)
print('Number of sentences after filtering out those with characters < 15:', len(assignments))

Number of sentences after filtering out those with characters < 15: 328091


## Regenerate revised_sentence_ind.json and revised_sentence_str.json with correct comment date

In [ ]:
jsonfilelist= {'nytimes': '/content/drive/My Drive/Colab Notebooks/media-agenda/data/media-agenda-dataset-quora/nytimes.json',
               'quora': '/content/drive/My Drive/Colab Notebooks/media-agenda/data/media-agenda-dataset-quora/quora.json', 
               'spiegel': '/content/drive/My Drive/Colab Notebooks/media-agenda/data/media-agenda-dataset-quora/spiegel.json'}


# en_global_ids = np.load('/content/drive/My Drive/Colab Notebooks/media-agenda/refactorization/data/en_sentence_global_id.npy')
# de_global_ids = np.load('/content/drive/My Drive/Colab Notebooks/media-agenda/refactorization/data/de_sentence_global_id.npy')
# reference_global_ids = np.concatenate((en_global_ids, de_global_ids))
# kmean_labels = np.load(directory + '15_kmean_labels.npy')

global_id = 0
sentences_index = []
sentences_string = []

for key in jsonfilelist:
  # load organic dataset downloaded from https://gitlab.lrz.de/social-rom/organic-dataset/media-agenda-dataset-quora
  with open(jsonfilelist[key], 'r') as f:
    corpus = json.load(f)
  print('begin: ', key)

  language = 'english' if key != Source.SPIEGEL else 'german'
  article_text_field = 'processed_article_text' if key != Source.QUORA else 'article_text' 
  if corpus != None:
    # each article
    article_id = 0
    for tmp_id, article in enumerate(corpus):
      # only keep articles with relevant == 1
      if 'relevant' in article and article['relevant'] == 1:
        # prepare metadata for article sentences
        if article[article_text_field] != '': 
          article_time = article['article_time']
          text = article[article_text_field]
          sentences = sent_tokenize(text, language)
          for s in sentences:
            sentences_index.append({'sentence_id': global_id, 
                                    'source': key,
                                    'article_id': article_id,
                                    'comment_id': -1, # assign -1 for indicating sentence from article
                                    'posting_time': article_time})
            sentences_string.append({str(global_id): s})
            global_id += 1
        # prepare metadata for comment sentences
        if article['comments'] != []:
          for comment_id, comment in enumerate(article['comments']):
            comment_time = comment['comment_time']
            text = comment['processed_comment_text']
            sentences = sent_tokenize(text, language)
            for s in sentences:
              sentences_index.append({'sentence_id': global_id, 
                                      'source': key,
                                      'article_id': article_id,
                                      'comment_id': comment_id,
                                      'posting_time': comment_time})
              sentences_string.append({str(global_id): s})
              global_id += 1
        # article_id increased by 1 after generating metadata for an article with relevant == 1
        article_id += 1
  print('Current number of sentence:', len(sentences_index))
  assert len(sentences_index) == len(sentences_string)
  assert sentences_string[:] == loaded_sentence_str[:len(sentences_string)]

util.list2json(sentences_index, '/content/drive/My Drive/Colab Notebooks/media-agenda/data/revised_all_sentences_ind.json')
util.list2json(sentences_string, '/content/drive/My Drive/Colab Notebooks/media-agenda/data/revised_all_sentences_str.json')

begin:  nytimes
Current number of sentence: 77023
begin:  quora
Current number of sentence: 136425
begin:  spiegel
Current number of sentence: 353786


Validation:

In [ ]:
assert len(sentences_index) == len(loaded_sentence_ind)

In [ ]:
print(sentences_index[0], loaded_sentence_ind[0])
print(sentences_index[70], loaded_sentence_ind[70])
print(sentences_index[157], loaded_sentence_ind[157])
print(sentences_index[160], loaded_sentence_ind[160])
print(sentences_index[485], loaded_sentence_ind[485])
print(sentences_index[77023], loaded_sentence_ind[77023])
print(sentences_index[136424], loaded_sentence_ind[136424])
print(sentences_index[136425], loaded_sentence_ind[136425])
print(sentences_index[353785], loaded_sentence_ind[353785])

{'sentence_id': 0, 'source': 'nytimes', 'article_id': 0, 'comment_id': -1, 'posting_time': '2005-11-01 00:00:00'} {'global_id': 0, 'corpus_name': 'nytimes', 'doc_id': 0, 'com_id': None}
{'sentence_id': 70, 'source': 'nytimes', 'article_id': 1, 'comment_id': -1, 'posting_time': '1970-09-07 00:00:00'} {'global_id': 70, 'corpus_name': 'nytimes', 'doc_id': 1, 'com_id': None}
{'sentence_id': 157, 'source': 'nytimes', 'article_id': 3, 'comment_id': 0, 'posting_time': '2008-04-18 12:58:00'} {'global_id': 157, 'corpus_name': 'nytimes', 'doc_id': 3, 'com_id': 0}
{'sentence_id': 160, 'source': 'nytimes', 'article_id': 3, 'comment_id': 2, 'posting_time': '2008-04-18 13:47:00'} {'global_id': 160, 'corpus_name': 'nytimes', 'doc_id': 3, 'com_id': 2}
{'sentence_id': 485, 'source': 'nytimes', 'article_id': 5, 'comment_id': -1, 'posting_time': '1997-12-15 00:00:00'} {'global_id': 485, 'corpus_name': 'nytimes', 'doc_id': 5, 'com_id': None}
{'sentence_id': 77023, 'source': 'quora', 'article_id': 0, 'comm

# Part 2

## Assign pre-predicted kmean cluster back to the sentences
* kmean clustering was carried out in NLP lab course, and
* only applied to sentences with character > 15

Filter out sentences with less than or equal to 15 characters and compare with desired sentence global id files generated in NLP lab course

In [ ]:
rsfile = '/content/drive/My Drive/Colab Notebooks/media-agenda/data/revised_all_sentences_str.json'
rifile = '/content/drive/My Drive/Colab Notebooks/media-agenda/data/revised_all_sentences_ind.json'

all_sentences = util.load_sentences(rsfile)
# replace the url with string 'url'
all_sentences = [util.replace_url(sentence) for sentence in all_sentences]
all_sentences = np.array(all_sentences)

nytimes_indeces = util.load_sentences_index(rifile, [Source.NYTIMES])
quora_indeces = util.load_sentences_index(rifile, [Source.QUORA])
spiegel_indeces = util.load_sentences_index(rifile, [Source.SPIEGEL])

# get sentences indeces (global indeces) for sentences whose length larger than 15 characters
nytimes_indeces_sm = [i for i, s in zip(nytimes_indeces, all_sentences[nytimes_indeces]) if len(s) > 15]
quora_indeces_sm = [i for i, s in zip(quora_indeces, all_sentences[quora_indeces]) if len(s) > 15]
spiegel_indeces_sm = [i for i, s in zip(spiegel_indeces, all_sentences[spiegel_indeces]) if len(s) > 15]

# define desired sentences
desired_sentence_ids = np.append(np.append(nytimes_indeces_sm, quora_indeces_sm), spiegel_indeces_sm)

print()
print('Number of sentences with length larger than 15 characters: ')
print('NYTIMES:', len(nytimes_indeces_sm))
print('QUORA:', len(quora_indeces_sm))
print('SPIEGEL:', len(spiegel_indeces_sm))
print('Total:', len(desired_sentence_ids))

en_global_ids = np.load('/content/drive/My Drive/Colab Notebooks/opinion-lab-group-2.3/refactorization/data/en_sentence_global_id.npy')
de_global_ids = np.load('/content/drive/My Drive/Colab Notebooks/opinion-lab-group-2.3/refactorization/data/de_sentence_global_id.npy')

np.testing.assert_array_equal(np.append(nytimes_indeces_sm, quora_indeces_sm), en_global_ids)
np.testing.assert_array_equal(spiegel_indeces_sm, de_global_ids)

Sentences file - loaded
Done - appended all sentences
Number of tokenized sentences from corpus: 353786
Sentences index file - loaded
Done - appended all sentences indeces
Number of indeces from corpus ['nytimes']: 77023
Sentences index file - loaded
Done - appended all sentences indeces
Number of indeces from corpus ['quora']: 59402
Sentences index file - loaded
Done - appended all sentences indeces
Number of indeces from corpus ['spiegel']: 217361

Number of sentences with length larger than 15 characters: 
NYTIMES: 73084
QUORA: 54380
SPIEGEL: 200627
Total: 328091


Save desired sentences and their corresponding sentence ID

In [ ]:
np.save('/content/drive/My Drive/Colab Notebooks/media-agenda/data/desired_sentence_ids.npy', desired_sentence_ids)

with open(rsfile, 'r') as f:
  sentences_dict = json.load(f)

sentences_dict = np.array(sentences_dict)[desired_sentence_ids]
util.list2json(sentences_dict, '/content/drive/My Drive/Colab Notebooks/media-agenda/data/sentence_dict.json')

Double check the sentences_dict is with desired sentences

In [ ]:
for i in range(353786):
  if i not in desired_sentence_ids:
    print(i)
    break

print(sentences_dict[19:26])
print(all_sentences[24])

24
[{'19': 'Ronnie Cummins, national director of the Organic Consumers Association, calls the initiative a "sneak attack engineered by the likes of Kraft, Dean Foods and Smucker\'s.".'}
 {'20': "One of the lobbyists for Altria, Kraft's majority owner, Abigail Blunt - the wife of Representative Roy Blunt, Republican of Missouri, who recently became interim House majority leader after Tom DeLay of Texas resigned from the post - has been working on the issue, the company says."}
 {'21': "Dean Foods' subsidiary Horizon Organic and the J. M. Smucker Company, the owner of Knudsen and Santa Cruz Organic juices, said they supported the work by the Organic Trade Association, which represents both large and small companies in the business, but did no lobbying on their own."}
 {'22': 'The amendment injects Congress directly into the debate over whether certain artificial ingredients and industrial chemicals should be allowed in products labeled organic.'}
 {'23': 'In a lawsuit ruled upon in Janua

Assign kmean result to corresponding desired sentences only (i.e. > 15 characters) 

In [ ]:
rifile = '/content/drive/My Drive/Colab Notebooks/media-agenda/data/revised_all_sentences_ind.json'
with open(rifile, 'r') as f:
  metadata = json.load(f)
  print('Sentences index file - loaded')


path = '/content/drive/My Drive/Colab Notebooks/media-agenda/refactorization/data/sentence_cluster_sentiment_dict.json'
with open(path, 'r') as f:
  assignments = json.load(f)
  print('Number of sentences after filtering out those with characters < 15:', len(assignments))


# kmean labels with 15 clusters is adopted
kmean_labels = np.load('/content/drive/My Drive/Colab Notebooks/opinion-lab-group-2.3/refactorization/data/15_kmean_labels.npy')
assert len(kmean_labels) == len(desired_sentence_ids)

desired_metadata_array = np.array(metadata)[desired_sentence_ids]

for i in range(len(desired_sentence_ids)):
  desired_metadata_array[i]['cluster'] = kmean_labels[i]
print('Completed with {} records'.format(len(desired_metadata_array)))


print()
print('Double check with original assignment (with incorrect comment posting time:')
pprint.pprint(np.array(assignments)[1000:1005])
pprint.pprint(desired_metadata_array[1000:1005])

pprint.pprint(np.array(assignments)[-5:])
pprint.pprint(desired_metadata_array[-5:])

util.list2json(desired_metadata_array, '/content/drive/My Drive/Colab Notebooks/media-agenda/data/sentence_cluster_dict.json')

Sentences index file - loaded
Number of sentences after filtering out those with characters < 15: 328091
Completed with 328091 records

Double check with original assignment (with incorrect comment posting time:
array([{'global_id': 1033, 'corpus_name': 'nytimes', 'doc_id': 16, 'com_id': 8, 'date': '2011-04-21', 'cluster': 6, 'sentiment': 0.0},
       {'global_id': 1034, 'corpus_name': 'nytimes', 'doc_id': 16, 'com_id': 8, 'date': '2011-04-21', 'cluster': 6, 'sentiment': 0.075},
       {'global_id': 1035, 'corpus_name': 'nytimes', 'doc_id': 16, 'com_id': 8, 'date': '2011-04-21', 'cluster': 6, 'sentiment': -0.375},
       {'global_id': 1036, 'corpus_name': 'nytimes', 'doc_id': 16, 'com_id': 8, 'date': '2011-04-21', 'cluster': 3, 'sentiment': 0.125},
       {'global_id': 1037, 'corpus_name': 'nytimes', 'doc_id': 16, 'com_id': 8, 'date': '2011-04-21', 'cluster': 3, 'sentiment': -0.0625}],
      dtype=object)
array([{'sentence_id': 1033, 'source': 'nytimes', 'article_id': 16, 'comment_id':

# Part 3

## Assign sentiments 
* sentiw_sentiment: SentiWordnet for English sentences and SentiWS for German sentences
* textblob_polarity and textblob_subjectivity: Textblob for English sentences and Textblob-de for German sentences



Load desired sentences and 

In [ ]:
sentences_dict_path = '/content/drive/My Drive/Colab Notebooks/media-agenda/data/sentence_dict.json'
sentences = util.load_sentences(sentences_dict_path)

path = '/content/drive/My Drive/Colab Notebooks/media-agenda/data/sentence_cluster_dict.json'
with open(path, 'r') as f:
  sentence_cluster_dict = json.load(f)
  print('Number of records in sentence-cluster dictionary', len(sentence_cluster_dict))


Sentences file - loaded
Done - appended all sentences
Number of tokenized sentences from corpus: 328091
Number of records in sentence-cluster dictionary 328091


### With SentiWordNet and SentiWS
* Use sentiment.py created by NLP lab course 

In [ ]:
tmp_list = []
for i in range(len(sentences)):
  sentence = sentences[i]
  language = 'en' if sentence_cluster_dict[i]['source'] != Source.SPIEGEL else 'de'

  score = sentiment.sentence_sentiment(str(sentence), language)
  sentence_cluster_dict[i]['sentiw_sentiment'] = score 
  tmp_list.append(score)
  if i in range(0, 10) or i in range(127464, 127474):
    print('{:<20} {} {} {}'.format(score, sentence_cluster_dict[i]['source'], language, sentence))

util.list2json(sentence_cluster_dict, '/content/drive/My Drive/Colab Notebooks/media-agenda/data/sentence_cluster_sentiment_dict.json')

0.125                nytimes en Customers at McDonald's restaurants in New England are about to get something a little different when they order coffee.
-0.125               nytimes en Through a deal with Green Mountain Coffee Roasters and Newman's Own, McDonald's will soon be serving a coffee that comes from organic beans and is certified Fair Trade because it meets higher standards in the treatment of coffee workers.
0.075                nytimes en The move, while still a test in a limited region, reflects a much broader trend: The growing interest among large food companies in offering organic foods along with their standard products.
0.5                  nytimes en General Mills markets the Cascadian Farms and Muir Glen brands; Kraft owns Back to Nature and Boca Foods, which makes soy burgers.
-0.03125             nytimes en Within the last few years, Dean Foods, the dairy giant, has acquired Horizon Organic and White Wave, maker of Silk organic soymilk.
0                    nytime

In [ ]:
pd.testing.assert_frame_equal(pd.DataFrame({'sentiment': tmp_list}), 
                              pd.DataFrame(pd.DataFrame(assignments).sentiment))

### With Textblob and Textblob-de

In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/media-agenda/data/sentence_cluster_sentiment_dict.json'
with open(path, 'r') as f:
  dictionary = json.load(f)
  print('Loaded {} records'.format(len(dictionary)))

print()
print('{:<25} {:<25} {:<8} {:<4} {}'.format('Score', 'Subjectivity', 'Source', '', 'Sentence'))
for i in range(len(sentences)):
  sentence = sentences[i]
  language = 'en' if dictionary[i]['source'] != Source.SPIEGEL else 'de'

  if language == 'en':
    sentiment = TextBlob(str(sentence)).sentiment
  elif language == 'de':
    sentiment = TextBlobDE(str(sentence)).sentiment
  
  score = sentiment.polarity
  subjectivity = sentiment.subjectivity

  dictionary[i]['textblob_polarity'] = score
  dictionary[i]['textblob_subjectivity'] = subjectivity

  if i in range(0, 10) or i in range(127464, 127474):
    print('{:<25} {:<25} {:<8} {:<4} {}'.format(score, subjectivity, sentence_cluster_dict[i]['source'], language, sentence))
    # print(dictionary[i])

print('Completed!')
util.list2json(dictionary, '/content/drive/My Drive/Colab Notebooks/media-agenda/data/sentence_cluster_sentiment_dict.json')

Loaded 328091 records

Score                     Subjectivity              Source        Sentence
-0.017045454545454548     0.5181818181818182        nytimes  en   Customers at McDonald's restaurants in New England are about to get something a little different when they order coffee.
0.33749999999999997       0.675                     nytimes  en   Through a deal with Green Mountain Coffee Roasters and Newman's Own, McDonald's will soon be serving a coffee that comes from organic beans and is certified Fair Trade because it meets higher standards in the treatment of coffee workers.
0.08571428571428572       0.19285714285714284       nytimes  en   The move, while still a test in a limited region, reflects a much broader trend: The growing interest among large food companies in offering organic foods along with their standard products.
0.02500000000000001       0.25                      nytimes  en   General Mills markets the Cascadian Farms and Muir Glen brands; Kraft owns Back to Natur

In [ ]:
dictionary[:3]

[{'article_id': 0,
  'cluster': 1,
  'comment_id': -1,
  'posting_time': '2005-11-01 00:00:00',
  'sentence_id': 0,
  'sentiw_sentiment': 0.125,
  'source': 'nytimes',
  'textblob_polarity': -0.017045454545454548,
  'textblob_subjectivity': 0.5181818181818182},
 {'article_id': 0,
  'cluster': 1,
  'comment_id': -1,
  'posting_time': '2005-11-01 00:00:00',
  'sentence_id': 1,
  'sentiw_sentiment': -0.125,
  'source': 'nytimes',
  'textblob_polarity': 0.33749999999999997,
  'textblob_subjectivity': 0.675},
 {'article_id': 0,
  'cluster': 3,
  'comment_id': -1,
  'posting_time': '2005-11-01 00:00:00',
  'sentence_id': 2,
  'sentiw_sentiment': 0.075,
  'source': 'nytimes',
  'textblob_polarity': 0.08571428571428572,
  'textblob_subjectivity': 0.19285714285714284}]

# Part 4 

## Generate required dataframe with date refinement before analysis

### Get dataframe for 2009 to 2017

In [ ]:
start_year = 2009
end_year = 2017
df = senti_util.get_sentence_cluster_sentiment_df(start_year = start_year, end_year = end_year, 
                                                  path = '/content/drive/My Drive/Colab Notebooks/media-agenda/data/sentence_cluster_sentiment_dict.json', verbose = True)

# file_name = 'df_' + str(start_year) + '_to_' + str(end_year) + '.csv'
# df.to_csv(file_name)

Loaded total 328091 cluster-sentiment assignments
Example of an assignment object

Number of article from:


,source,num_of_article
0,nytimes,327
1,quora,1338
2,spiegel,152
3,total,1817



Unique cluster after filtering:
array([ 1,  3, 10, 12,  6, 13,  8, 11,  0,  5,  7])

Min year after filtering: 2009
Max yearafter filtering: 2017
Number of sentences after filtering: 211443


,Source,Total sentences,Article sentences,Comment sentences
0,nytimes,50002,8004,41998
1,quora,44140,1022,43118
2,spiegel,117301,5157,112144



Display top 5 row of the preprocessed dataframe:


,index,sentence_id,source,article_id,comment_id,posting_time,cluster,sentiw_sentiment,textblob_polarity,textblob_subjectivity,is_comment,date,month,year
0,132,134,nytimes,2,-1,2016-04-25,1,-0.031250,0.050000,0.200000,False,2016-04-25,4,2016
1,133,135,nytimes,2,-1,2016-04-25,7,0.208333,0.175000,0.566667,False,2016-04-25,4,2016
2,136,138,nytimes,2,-1,2016-04-25,12,0.000000,0.000000,0.000000,False,2016-04-25,4,2016
3,137,139,nytimes,2,-1,2016-04-25,12,-0.375000,-0.263889,0.632937,False,2016-04-25,4,2016
4,138,140,nytimes,2,-1,2016-04-25,12,0.156250,-0.100000,0.366667,False,2016-04-25,4,2016



Reference:
Table showing the articles with posting time updated 


,source,article_id,changed from,to
0,nytimes,310,2014-02-06 00:00:00,2012-09-11
1,quora,331,2014-06-09 00:16:00,2013-08-01
2,quora,736,2016-01-18 17:54:00,2015-08-25
3,quora,852,2015-08-21 04:18:00,2012-06-13
4,spiegel,39,2008-08-20 11:24:52,2008-08-19
5,spiegel,46,2009-07-31 16:01:55,2009-07-10
6,spiegel,54,2007-12-22 08:22:24,2007-08-28
7,spiegel,56,2007-06-12 18:54:41,2007-06-11
8,spiegel,59,2009-02-21 12:36:23,2008-11-13
9,spiegel,62,2008-11-25 06:04:32,2008-11-13



Current values for the 12th article above:
Source: spiegel  Article_id: 78


,index,sentence_id,source,article_id,comment_id,posting_time,cluster,sentiw_sentiment,textblob_polarity,textblob_subjectivity,is_comment,date,month,year
145155,222919,239782,spiegel,78,-1,2011-01-04,3,0.0000,0.000000,0.000000,False,2011-01-04,1,2011
145156,222920,239783,spiegel,78,-1,2011-01-04,13,0.0000,0.000000,1.000000,False,2011-01-04,1,2011
145157,222921,239784,spiegel,78,-1,2011-01-04,11,0.0040,0.000000,0.000000,False,2011-01-04,1,2011
145158,222922,239785,spiegel,78,-1,2011-01-04,13,-0.6621,-0.700000,0.000000,False,2011-01-04,1,2011
145159,222923,239786,spiegel,78,-1,2011-01-04,12,-0.3233,-0.666667,0.333333,False,2011-01-04,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145217,222985,239848,spiegel,78,-1,2011-01-04,10,0.0000,0.000000,0.000000,False,2011-01-04,1,2011
145218,222986,239849,spiegel,78,-1,2011-01-04,3,0.0000,0.000000,0.000000,False,2011-01-04,1,2011
145219,222987,239850,spiegel,78,-1,2011-01-04,12,-0.0048,-0.700000,0.000000,False,2011-01-04,1,2011
145220,222988,239851,spiegel,78,-1,2011-01-04,12,0.0000,0.000000,0.000000,False,2011-01-04,1,2011


### Hash the posting time with different desired time interval

Desired time interval can be 2 days, 4 days, 7 days, 14 days, 28 days etc.






In [ ]:
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df, start_year, end_year, interval = '2D', column_name = 'bin_of_every_2_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '3D', column_name = 'bin_of_every_3_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '4D', column_name = 'bin_of_every_4_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '5D', column_name = 'bin_of_every_5_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '6D', column_name = 'bin_of_every_6_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '7D', column_name = 'bin_of_every_7_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '14D', column_name = 'bin_of_every_14_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '28D', column_name = 'bin_of_every_28_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '56D', column_name = 'bin_of_every_56_days')
print('Completed!')

file_name = 'df_' + str(start_year) + '_to_' + str(end_year) + '_with_time_interval_indexing.csv'
df_new.to_csv(file_name)

CPU times: user 16min 35s, sys: 143 ms, total: 16min 35s
Wall time: 16min 36s
CPU times: user 8min 25s, sys: 65.9 ms, total: 8min 25s
Wall time: 8min 26s
CPU times: user 4min 39s, sys: 64.9 ms, total: 4min 39s
Wall time: 4min 40s
Completed!


### Additional hash

In [ ]:
start_year = 2009
end_year = 2017
start_datetime, end_datetime, _, _ = util.get_start_end_datetime(start_year, end_year)
df = senti_util.get_sentence_cluster_sentiment_df(start_year = start_year, end_year = end_year, 
                                                  path = '/content/drive/My Drive/Colab Notebooks/media-agenda/data/sentence_cluster_sentiment_dict.json', verbose = False)

display(df.head(5))
display(df.tail(5))

%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df, start_year, end_year, interval = '56D', column_name = 'bin_of_every_56_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '84D', column_name = 'bin_of_every_84_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '112D', column_name = 'bin_of_every_112_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '140D', column_name = 'bin_of_every_140_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '168D', column_name = 'bin_of_every_168_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '196D', column_name = 'bin_of_every_196_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '224D', column_name = 'bin_of_every_224_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '252D', column_name = 'bin_of_every_252_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '280D', column_name = 'bin_of_every_280_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '308D', column_name = 'bin_of_every_308_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '336D', column_name = 'bin_of_every_336_days')
%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df_new, start_year, end_year, interval = '364D', column_name = 'bin_of_every_364_days')
print('Completed!')
file_name = 'df_' + str(start_year) + '_to_' + str(end_year) + '_with_time_interval_indexing_new.csv'
df_new.to_csv(file_name)

,index,sentence_id,source,article_id,comment_id,posting_time,cluster,sentiw_sentiment,textblob_polarity,textblob_subjectivity,is_comment,date,month,year
0,132,134,nytimes,2,-1,2016-04-25,1,-0.031250,0.050000,0.200000,False,2016-04-25,4,2016
1,133,135,nytimes,2,-1,2016-04-25,7,0.208333,0.175000,0.566667,False,2016-04-25,4,2016
2,136,138,nytimes,2,-1,2016-04-25,12,0.000000,0.000000,0.000000,False,2016-04-25,4,2016
3,137,139,nytimes,2,-1,2016-04-25,12,-0.375000,-0.263889,0.632937,False,2016-04-25,4,2016
4,138,140,nytimes,2,-1,2016-04-25,12,0.156250,-0.100000,0.366667,False,2016-04-25,4,2016


,index,sentence_id,source,article_id,comment_id,posting_time,cluster,sentiw_sentiment,textblob_polarity,textblob_subjectivity,is_comment,date,month,year
211438,328049,353744,spiegel,150,-1,2009-05-07 11:35:39,1,0.004000,0.000000,0.000000,False,2009-05-07,5,2009
211439,328050,353745,spiegel,150,-1,2009-05-07 11:35:39,1,0.000000,0.000000,0.000000,False,2009-05-07,5,2009
211440,328051,353746,spiegel,150,-1,2009-05-07 11:35:39,1,0.000000,0.000000,0.000000,False,2009-05-07,5,2009
211441,328052,353747,spiegel,150,-1,2009-05-07 11:35:39,12,0.004000,1.000000,0.000000,False,2009-05-07,5,2009
211442,328053,353748,spiegel,150,-1,2009-05-07 11:35:39,3,0.027114,0.333333,0.166667,False,2009-05-07,5,2009


CPU times: user 15min 3s, sys: 129 ms, total: 15min 3s
Wall time: 15min 4s
CPU times: user 10min 21s, sys: 84.9 ms, total: 10min 21s
Wall time: 10min 22s
CPU times: user 8min 5s, sys: 50 ms, total: 8min 5s
Wall time: 8min 5s
CPU times: user 6min 44s, sys: 67 ms, total: 6min 44s
Wall time: 6min 44s
CPU times: user 5min 48s, sys: 62.9 ms, total: 5min 48s
Wall time: 5min 48s
CPU times: user 5min, sys: 38 ms, total: 5min
Wall time: 5min
CPU times: user 4min 35s, sys: 37 ms, total: 4min 35s
Wall time: 4min 35s
CPU times: user 4min 6s, sys: 39.9 ms, total: 4min 6s
Wall time: 4min 6s
CPU times: user 3min 47s, sys: 29 ms, total: 3min 47s
Wall time: 3min 48s
CPU times: user 3min 34s, sys: 32 ms, total: 3min 34s
Wall time: 3min 34s
CPU times: user 3min 24s, sys: 46 ms, total: 3min 24s
Wall time: 3min 24s
CPU times: user 3min 17s, sys: 40 ms, total: 3min 17s
Wall time: 3min 17s
Completed!


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/media-agenda/data/df_2009_to_2017_with_time_interval_indexing.csv')
df = df.drop(['bin_of_every_56_days'], axis=1)
# len(df.columns)
to_save_df = df.merge(df_new[['sentence_id', 'bin_of_every_56_days',
                 'bin_of_every_84_days', 'bin_of_every_112_days', 'bin_of_every_140_days', 
                 'bin_of_every_168_days', 'bin_of_every_196_days', 'bin_of_every_224_days', 
                 'bin_of_every_252_days', 'bin_of_every_280_days', 'bin_of_every_308_days',
                 'bin_of_every_336_days', 'bin_of_every_364_days']], how = 'left', on = 'sentence_id', validate = 'one_to_one')

# file_name = 'df_' + str(start_year) + '_to_' + str(end_year) + '_with_time_interval_indexing_3.csv'
# to_save_df.to_csv(file_name)

In [ ]:
# add 'bin_of_every_1_day' column
date_list = senti_util.get_date_list(start_datetime, end_datetime, freq = '1D')
date_df = pd.DataFrame({'bin_of_every_1_day': range(len(date_list)),
                        'date': date_list})
date_df['date'] = date_df['date'].dt.strftime('%Y-%m-%d')
to_save_df = to_save_df.merge(date_df, how = 'left', on = 'date')

# file_name = 'df_' + str(start_year) + '_to_' + str(end_year) + '_with_time_interval_indexing_final.csv'
# to_save_df.to_csv(file_name)

In [3]:
start_year = 2009
end_year = 2017
start_datetime, end_datetime, _, _ = util.get_start_end_datetime(start_year, end_year)
df = senti_util.get_sentence_cluster_sentiment_df(start_year = start_year, end_year = end_year, 
                                                  path = '/content/drive/My Drive/Colab Notebooks/media-agenda/data/sentence_cluster_sentiment_dict.json', verbose = False)

display(df.head(5))
display(df.tail(5))

%time df_new, _ = senti_util.get_df_with_time_interval_indexing(df, start_year, end_year, interval = '182D', column_name = 'bin_of_every_182_days')

current_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/media-agenda/data/df_2009_to_2017_with_time_interval_indexing.csv')

to_save_df = current_df.merge(df_new[['sentence_id', 'bin_of_every_182_days']], how = 'left', on = 'sentence_id', validate = 'one_to_one')

file_name = 'df_' + str(start_year) + '_to_' + str(end_year) + '_with_time_interval_indexing_5.csv'
to_save_df.to_csv(file_name)

,index,sentence_id,source,article_id,comment_id,posting_time,cluster,sentiw_sentiment,textblob_polarity,textblob_subjectivity,is_comment,date,month,year
0,132,134,nytimes,2,-1,2016-04-25,1,-0.031250,0.050000,0.200000,False,2016-04-25,4,2016
1,133,135,nytimes,2,-1,2016-04-25,7,0.208333,0.175000,0.566667,False,2016-04-25,4,2016
2,136,138,nytimes,2,-1,2016-04-25,12,0.000000,0.000000,0.000000,False,2016-04-25,4,2016
3,137,139,nytimes,2,-1,2016-04-25,12,-0.375000,-0.263889,0.632937,False,2016-04-25,4,2016
4,138,140,nytimes,2,-1,2016-04-25,12,0.156250,-0.100000,0.366667,False,2016-04-25,4,2016


,index,sentence_id,source,article_id,comment_id,posting_time,cluster,sentiw_sentiment,textblob_polarity,textblob_subjectivity,is_comment,date,month,year
211438,328049,353744,spiegel,150,-1,2009-05-07 11:35:39,1,0.004000,0.000000,0.000000,False,2009-05-07,5,2009
211439,328050,353745,spiegel,150,-1,2009-05-07 11:35:39,1,0.000000,0.000000,0.000000,False,2009-05-07,5,2009
211440,328051,353746,spiegel,150,-1,2009-05-07 11:35:39,1,0.000000,0.000000,0.000000,False,2009-05-07,5,2009
211441,328052,353747,spiegel,150,-1,2009-05-07 11:35:39,12,0.004000,1.000000,0.000000,False,2009-05-07,5,2009
211442,328053,353748,spiegel,150,-1,2009-05-07 11:35:39,3,0.027114,0.333333,0.166667,False,2009-05-07,5,2009


CPU times: user 5min 30s, sys: 16.8 ms, total: 5min 30s
Wall time: 5min 30s


In [ ]:
# a = pd.read_csv('/content/drive/My Drive/Colab Notebooks/media-agenda/data/df_2009_to_2017_with_time_interval_indexing_5.csv')
# a
# # a = a.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
# # a
# # file_name = 'df_' + str(start_year) + '_to_' + str(end_year) + '_with_time_interval_indexing_5.csv'
# # a.to_csv(file_name)